In [3]:
import sqlite3
import json

# Load JSON
with open("../data/cochrane_parsed.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Connect to SQLite DB 
conn = sqlite3.connect("../database/cochrane_reviews.db")
cur = conn.cursor()

# Create tables
cur.executescript("""
DROP TABLE IF EXISTS reviews;
DROP TABLE IF EXISTS authors;
DROP TABLE IF EXISTS abstracts;
DROP TABLE IF EXISTS pico;

CREATE TABLE reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    source_url TEXT
);

CREATE TABLE authors (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    review_id INTEGER,
    name TEXT,
    FOREIGN KEY (review_id) REFERENCES reviews(id)
);

CREATE TABLE abstracts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    review_id INTEGER,
    section TEXT,
    content TEXT,
    FOREIGN KEY (review_id) REFERENCES reviews(id)
);

CREATE TABLE pico (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    review_id INTEGER,
    category TEXT,
    term TEXT,
    FOREIGN KEY (review_id) REFERENCES reviews(id)
);
""")

# Insert data
for review in data:
    title = review.get("title")
    url = review.get("source_url")

    # Insert review and get its ID
    cur.execute("INSERT INTO reviews (title, source_url) VALUES (?, ?)", (title, url))
    review_id = cur.lastrowid

    # Insert authors
    for author in review.get("authors", []):
        cur.execute("INSERT INTO authors (review_id, name) VALUES (?, ?)", (review_id, author))

    # Insert abstract sections
    for section, content in review.get("abstract", {}).items():
        cur.execute("INSERT INTO abstracts (review_id, section, content) VALUES (?, ?, ?)",
                    (review_id, section, content))

    # Insert PICO terms
    for category, terms in review.get("pico", {}).items():
        for term in terms:
            cur.execute("INSERT INTO pico (review_id, category, term) VALUES (?, ?, ?)",
                        (review_id, category, term))

# Commit and close
conn.commit()
conn.close()
print("Data migration to SQLite complete")


Data migration to SQLite complete
